In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

######################
## Helper Functions ##
######################
def get_frame(video_path, frame_idx, width, height):
    """
    Grad frame from video at frame_idx.
    
    @param video_path: path of video to read
    @param frame_idx: index of frame to grab
    
    @return the specified frame at frame_idx
    """
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    frame = cv2.resize(frame, (width, height))
    return frame

def read_data(labels_path, width, height):
    """
    Read video path and label located at labels_path.
    
    @param labels_path: path to labels text
    @param width: resize image to specified width
    @param height: resize image to specified height
    
    @return X: formatted data in form [instances x features]
    @return y: formatted data in form [instances x 1]
    """
    import os.path
    if os.path.isfile("X.npy") and os.path.isfile("y.npy"):
        X = np.load("X.npy")
        y = np.load("y.npy")
        return X, y
    else:
        print("Creating dataset...")
        X = []
        y = []
        with open(labels_path, "r") as file:
            for line in file:
                video_path, video_label = line.split()
                frame = get_frame(video_path, 90, width, height)
                frame = np.reshape(frame, (width*height*3))
                X.append(frame)
                y.append(int(video_label))
        X = np.array(X)
        y = np.array(y)
        np.save("X.npy", X)
        np.save("y.npy", y)
        return X, y

def split_data(X, y, split_ratio):
    """
    Split data into training and testing.
    
    @param X: input data [instances x features]
    @param y: input labels [instances x 1]
    @param split: ratio of training to testing instances
                    @pre [0-1]
    
    @return: training and testing data and labels
    """
    rand_idx = np.random.permutation(X.shape[0])
    idx = int(X.shape[0] * split_ratio)
    Xtr = X[rand_idx[:idx]]
    ytr = y[rand_idx[:idx]]
    Xte = X[rand_idx[idx:]]
    yte = y[rand_idx[idx:]]
    return Xtr, ytr, Xte, yte

In [38]:
class NearestNeighbor(object):
    def __init__(self):
        pass
    
    def train(self, X, y):
        """
        Train nearest neighbor classifier (i.e. store training examples).
        
        @param X: training data [instances x features]
        @param y: training labels [instances x 1]
        
        """
        # nearest neighbor classifier simply remembers all training data
        self.Xtr = X
        self.ytr = y
        
    def predict(self, X, K):
        """
        Predict labels on X.
        
        @param X: input data to predict labels [instances x features]
        @param K: number of nearest neighbors
        
        @return ypred: predicted labels [instances x 1]
        """
        num_test = X.shape[0]
        num_train = self.Xtr.shape[0]
        # set output type to match input type
        y_pred = np.zeros(num_test, dtype=self.ytr.dtype)
        dists = np.zeros((num_test, num_train))
        # find L2 distance between all test and training points
        dists = -2 * np.dot(X, self.Xtr.T)
        dists = np.sum(self.Xtr**2, axis=1) + dists
        dists = (dists.T + np.sum(X**2, axis=1)).T
        # predict label
        for i in range(num_test):
            min_indices = np.argpartition(dists[i, :], K)[:K]
            closest_y = self.ytr[min_indices]
            y_pred[i] = np.argmax(np.bincount(closest_y) + 1)
        return y_pred
    
    def accuracy(self, X, y, K):
        """
        Find accuracy on input data and labels.
        
        @param X: input data to predict labels [instances x features]
        @param K: number of nearest neighbors
        
        @return acc: resulting accuracy
        """
        ypred = self.predict(X, K)
        acc = np.mean(ypred == y) * 100
        return acc

In [40]:
# Create training and testing data
labels_path = "labels_gary.txt"
X, y = read_data(labels_path, 100, 100)
Xtr, ytr, Xte, yte = split_data(X, y, 0.6)
print("Training data dimensions:", Xtr.shape, ytr.shape)
print("Testing data dimensions:", Xte.shape, yte.shape)

# run nearest neighbor
nn = NearestNeighbor()
nn.train(Xtr, ytr)
acc = nn.accuracy(Xte, yte, 10)
K = [1, 10, 50, 100]
acc = []
print("Computing accuracies...")
for k in K:
    acc.append(nn.accuracy(Xte, yte, k))
print("Finished Training.")

Training data dimensions: (1050, 30000) (1050,)
Testing data dimensions: (700, 30000) (700,)
Computing accuracies...
Finished Training.
